In [ ]:
import re
import dns.resolver
import smtplib

In [ ]:

def is_valid_email_format(email):
    regex = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    return re.match(regex, email) is not None

def get_mx_records(domain):
    try:
        mx_records = dns.resolver.resolve(domain, 'MX')
        return [record.exchange.to_text() for record in mx_records]
    except Exception as e:
        print(f"Error retrieving MX records: {e}")
        return None

def verify_email(email):
    if not is_valid_email_format(email):
        print("Invalid email format")
        return False
    
    domain = email.split('@')[1]
    mx_records = get_mx_records(domain)
    if not mx_records:
        print("No MX records found for domain")
        return False
    
    # Try connecting to each MX server
    for mx in mx_records:
        try:
            server = smtplib.SMTP(mx)
            server.set_debuglevel(0)
            server.helo()
            server.mail('test@example.com')
            code, message = server.rcpt(email)
            server.quit()
            if code == 250:
                return True
        except Exception as e:
            print(f"Error verifying email: {e}")
    
    return False

In [ ]:

# Example usage
email = "example@example.com"
if verify_email(email):
    print(f"The email {email} exists.")
else:
    print(f"The email {email} does not exist.")